In [1]:
import pandas as pd

In [4]:
df = pd.read_pickle('SNIP_train_logs.pkl')
df

,architecture,sparsity_ratio,max_epochs,dataset,sanity_check,epoch,loss,train_acc,test_acc
0,resnet32,0.6,160,cifar10,corrupted_data,0,0.0265504,47.06,48.11
1,resnet32,0.6,160,cifar10,corrupted_data,1,0.0172153,67.81,66.57
2,resnet32,0.6,160,cifar10,corrupted_data,2,0.0128052,73.12,72.44
3,resnet32,0.6,160,cifar10,corrupted_data,3,0.0104739,75.30,74.62
4,resnet32,0.6,160,cifar10,corrupted_data,4,0.0091724,77.77,76.75
...,...,...,...,...,...,...,...,...,...
6974,vgg19,0.98,160,cifar100,original,155,0.0573957,18.34,14.26
6975,vgg19,0.98,160,cifar100,original,156,0.0573664,18.40,14.28
6976,vgg19,0.98,160,cifar100,original,157,0.0573842,18.56,14.22
6977,vgg19,0.98,160,cifar100,original,158,0.0573920,18.54,14.18


In [44]:
def locate(architecture, sparsity_ratio, dataset, sanity_check):
    return df.loc[
        (df['architecture'] == architecture) & 
        (df['sparsity_ratio'] == str(sparsity_ratio)) &
        (df['dataset'] == dataset) & 
        (df['sanity_check'] == sanity_check)
    ]

data = {}
reference_points = sorted(map(float, df['sparsity_ratio'].unique()))[1:] # There are results for 0 we don't want now
data['reference points'] = reference_points
for model in ['ResNet32', 'VGG19']:
    data[model] = {}
    architecture = model.lower()
    for sanity_check in ['original', 'corrupted_data', 'layerwise_rearrange']:
        sanity_check_fancy = sanity_check.replace('_', ' ')
        data[model][sanity_check_fancy] = []
        for ref_point in reference_points:
            values = map(float, locate(architecture, ref_point, 'cifar10', sanity_check)['test_acc'].unique())
            data[model][sanity_check_fancy].append(max(values))
            


{'reference points': [0.4, 0.6, 0.8, 0.9, 0.95, 0.98],
 'ResNet32': {'original': [94.52, 94.84, 93.95, 92.93, 91.31, 87.97],
  'corrupted data': [94.72, 94.26, 94.05, 92.79, 91.15, 88.02],
  'layerwise rearrange': [94.62, 94.32, 93.75, 92.89, 91.65, 88.0]},
 'VGG19': {'original': [93.61, 93.54, 93.24, 93.43, 93.31, 89.97],
  'corrupted data': [93.2, 93.31, 93.57, 93.54, 93.0, 49.13],
  'layerwise rearrange': [93.36, 93.51, 93.18, 93.48, 93.33, 88.48]}}